# Assignment 01: Evaluate the Spam Collection Dataset



#### 1: Import the dataset

In [1]:
#Import the required libraries
import pandas as pd
import string
from nltk.corpus import stopwords
import nltk
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [8]:
#Import the spam collection dataset
spamdataset = pd.read_csv("/content/SpamCollection",sep='\t',names=['comment','label'])

#### 2: Analyze the dataset

In [9]:
#View the first five records of the dataset
spamdataset.head(5)

,comment,label
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


#### 3: View additional information about the spam data

In [10]:
#View additional information
spamdataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5572 entries, 0 to 5571
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   comment  5572 non-null   object
 1   label    5572 non-null   object
dtypes: object(2)
memory usage: 87.2+ KB


In [15]:
#View information about messages based on how they are labeled (ham or spam) (Hint: first group them based on their response label, then describe them)
spamdataset.groupby("comment").describe()

length                                   
          count mean  std  min  25%  50%  75%  max
comment                                           
ham      4825.0  3.0  0.0  3.0  3.0  3.0  3.0  3.0
spam      747.0  4.0  0.0  4.0  4.0  4.0  4.0  4.0

#### 4: View the length of messages

In [16]:
#Calculate the length of the messages and add these values as a new column in the dataset
spamdataset['length'] =spamdataset['label'].apply(len)

In [17]:
#Verify if the new column and its values are added to the dataset
spamdataset.head(5)

,comment,label,length
0,ham,"Go until jurong point, crazy.. Available only ...",111
1,ham,Ok lar... Joking wif u oni...,29
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,155
3,ham,U dun say so early hor... U c already then say...,49
4,ham,"Nah I don't think he goes to usf, he lives aro...",61


#### 5: Define a function to eliminate stopwords

In [22]:
#Define the required function
def message_text_process(mess):
    # Check characters to see if there are punctuations
    no_punctuation = [char for char in mess if char not in string.punctuation]
    # Form the sentence
    no_punctuation = ''.join(no_punctuation)
    # Eliminate any stopwords
    return [word for word in no_punctuation.split() if word.lower() not in stopwords.words('english')]

In [23]:
#Process the text using a vectorizer
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()

#### 6: Apply Bag of Words

In [24]:
#Use Bag of Words and fit the data (messages) into it

from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd

content = """Cake is a form of sweet food made from flour, sugar, and other ingredients, that is usually baked.
In their oldest forms, cakes were modifications of bread, but cakes now cover a wide range of preparations that can be simple or elaborate, and that share features with other desserts such as pastries, meringues, custards, and pies."""

count_vectorizer = CountVectorizer()

bag_of_words = count_vectorizer.fit_transform(content.splitlines())

pd.DataFrame(bag_of_words.toarray(), columns = count_vectorizer.get_feature_names())

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,and,as,baked,be,bread,but,cake,cakes,can,cover,...,simple,such,sugar,sweet,that,their,usually,were,wide,with
0,1,0,1,0,0,0,1,0,0,0,...,0,0,1,1,1,0,1,0,0,0
1,2,1,0,1,1,1,0,2,1,1,...,1,1,0,0,2,1,0,1,1,1


In [27]:
#Print the length of Bag of Words stored in the vocabulary_ attribute
import string
from nltk.corpus import stopwords
bag_of_words = CountVectorizer(analyzer=message_text_process).fit(spamdataset['label'])

In [29]:
#Store Bag of Words for messages using the transform() method
label_bagofwords = bag_of_words.transform(spamdataset['label'])

#### 7: Apply the tf-idf transformer

In [31]:
#Apply the tf-idf transformer and fit the bag of words into it (transformed version)
from sklearn.feature_extraction.text import TfidfTransformer
tfidf_transformer = TfidfTransformer().fit(label_bagofwords)

In [32]:
#Print the shape of the transformer 
label_tfidf = tfidf_transformer.transform(label_bagofwords)
print(label_tfidf.shape)

(5572, 11425)


#### 8: Detect spam with the Naïve Bayes model

In [44]:
#Create a Naïve Bayes model to detect the spam and fit the tf-idf data into it
from sklearn.naive_bayes import MultinomialNB
spam_detection_model = MultinomialNB().fit(label_tfidf,spamdataset['comment'])

In [45]:
#Evaluate the model's accuracy by comparing the predicted and expected values for, say, message#2 and message#5
print('expected email type:', spamdataset.comment[2])

expected email type: spam


In [48]:
print('expected email type: ', spam_detection_model.predict(label_tfidf)[2])

expected email type:  spam
